# Payoff Matrices (part 2)

> This module contains payoff matrices for different evolutionary games
>
> Part 2 contains payoff matrices for the following games
> - Encanacao et al. 2016
> - Vasconcelos et al. 2014
> - Stochastic payoffs, a. la. Hilbe et al. 2018

In [1]:
# | default_exp `dummy_exp`


In [2]:
# | hide
# We could like to export all payoff matrices to the same module, `payoffs`.
# However,  we must set a default_exp and if we use the same module as
# another notebook, we will overwrite all existing exports with those in
# this file. Hence we set a dummy default_exp, `dummy_exp`, and each time
# we export a cell, we specify the `payoffs` module.


In [3]:
# | hide
# | export
from nbdev.showdoc import *
from fastcore.test import test_eq, test_close
import collections
import functools
from gh_pages_example.utils import *
from gh_pages_example.types import *
from gh_pages_example.methods import *
from gh_pages_example.model_utils import *
import itertools
import math
import typing

import fastcore.test
import more_itertools
import numpy as np
import nptyping


/mnt/d/Paolo-Work/git/gh-pages-example/gh_pages_example/model_utils.py:299: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (ind not in allowed_inds) and (str(ind) not in allowed_inds):
/mnt/d/Paolo-Work/git/gh-pages-example/gh_pages_example/methods.py:260: ComplexWarning: Casting complex values to real discards the imaginary part
  ergodic = np.array(V.transpose(0, 2, 1)[y], dtype=float)


In [4]:
np.set_printoptions(suppress=True)  # don't use scientific notation


In [5]:
# | export payoffs
def payoffs_encanacao_2016(models):
    names = ['b_r', 'b_s', 'c_s', 'c_t', 'σ']
    b_r, b_s, c_s, c_t, σ = [models[k] for k in names]
    payoffs = {}
    n_players = 3
    n_sectors = 3
    n_strategies_per_sector = [2, 2, 2]
    n_strategies_total = 6
    # All players are from the first sector, playing that sector's first strategy
    index_min = "0-0-0"
    # All players are from the third sector, playing that sector's second strategy
    index_max = "5-5-5"
    # Note: The seperator makes it easy to represent games where n_strategies_total >= 10.

    # It is also trivial to define a vector which maps these indexes to strategy profiles
    # As sector order is fixed we could neglect to mention suscripts for each sector
    strategy_names = ["D", "C", "D", "C", "D", "C"]

    zero = np.zeros(b_r.shape[0])
    # As in the main text
    payoffs["C-C-C"] = {"P3": b_r-2*c_s,
                        "P2": σ+b_s-c_t,
                        "P1": σ+b_s}
    payoffs["C-C-D"] = {"P3": -c_s,
                        "P2": b_s-c_t,
                        "P1": zero}
    payoffs["C-D-C"] = {"P3": b_r-c_s,
                        "P2": zero,
                        "P1": b_s}
    payoffs["C-D-D"] = {"P3": zero,
                        "P2": σ,
                        "P1": σ}
    payoffs["D-C-C"] = {"P3": zero,
                        "P2": σ-c_t,
                        "P1": σ}
    payoffs["D-C-D"] = {"P3": zero,
                        "P2": -c_t,
                        "P1": zero}
    payoffs["D-D-C"] = {"P3": zero,
                        "P2": zero,
                        "P1": zero}
    payoffs["D-D-D"] = {"P3": zero,
                        "P2": σ,
                        "P1": σ}

    # The following indexes capture all strategy profiles where each player is fixed to a unique sector
    # (and player order does not matter, so we need only consider one ordering of sectors).
    payoffs["4-2-0"] = payoffs["D-D-D"]
    payoffs["4-2-1"] = payoffs["D-D-C"]
    payoffs["4-3-0"] = payoffs["D-C-D"]
    payoffs["4-3-1"] = payoffs["D-C-C"]
    payoffs["5-2-0"] = payoffs["C-D-D"]
    payoffs["5-2-1"] = payoffs["C-D-C"]
    payoffs["5-3-0"] = payoffs["C-C-D"]
    payoffs["5-3-1"] = payoffs["C-C-C"]
    return {**models, "payoffs": payoffs}


## Vasconselos et al. 2014

They introduce a model of a Collective Risk Dilemma. It is a variant of the
public goods game where players must achieve a target level of contributions
to avoid risking a disaster which destroys the group's endowments.

We compute payoffs when players contribute $0$ or a fixed $c$ proportion of
their endowment as a contribution in
a game with up to $n$ participants. To do this, we compute the payoffs as a
function of the number of contributors, then use that function for each
relevant strategy profile.

In [6]:
# | export payoffs
@multi
def build_payoffs(models: dict):
    return models.get('payoffs_key')


@method(build_payoffs, 'vasconcelos_2014_primitives')
def build_payoffs(models: dict):
    names = ['payoffs_state', 'c', 'T', 'b_r', 'b_p', 'r']
    payoffs_state, c, T, b_r, b_p, r = [models[k] for k in names]
    strategy_counts = payoffs_state['strategy_counts']
    n_r = strategy_counts["2"]
    n_p = strategy_counts["4"]
    risk = r * (n_r * c * b_r + n_p * c * b_p < T)
    # The payoffs must be computed for each strategy type in the interaction.
    # In games where we employ hypergeometric sampling, we usually do not
    # care about player order in the interaction. If order did matter, then
    # we would represent the payoffs per strategy still but it would capture
    # the expected payoffs given how likely a player of that strategy was to
    # play in each node of the extensive-form game. Non-players of type 0
    # usually do not have payoffs.
    payoffs = {"1": (1 - risk) * b_r,  # rich_free_rider
               "2": (1 - risk) * c * b_r,  # rich_contributor
               "3": (1 - risk) * b_p,  # poor_free_rider
               "4": (1 - risk) * c * b_p}  # poor_contributor
    return {**models, "payoff_primitives": payoffs}


@method(build_payoffs, 'vasconcelos_2014')
def build_payoffs(models: dict):
    profiles = create_profiles({'n_players': models.get('n_players', 5),
                                'n_strategies': [2, 2]})['profiles']
    payoffs = {}
    for profile in profiles:
        profile_tuple = thread_macro(profile,
                                     (str.split, "-"),
                                     (map, int, "self"),
                                     list,
                                     reversed,
                                     list,
                                     np.array,
                                     )
        strategy_counts = {f"{i}": np.sum(
            profile_tuple == i) for i in range(5)}
        payoffs_state = {'strategy_counts': strategy_counts}
        primitives = thread_macro(models,
                                  (assoc,
                                   'payoffs_state', payoffs_state,
                                   'payoffs_key', "vasconcelos_2014_primitives"),
                                  build_payoffs,
                                  (get, "payoff_primitives"),
                                  )
        payoffs[profile] = {}
        for i, strategy in enumerate(profile_tuple):
            if strategy == 0:
                continue
            elif strategy == 1:
                payoffs[profile][f"P{i+1}"] = primitives['1']
            elif strategy == 2:
                payoffs[profile][f"P{i+1}"] = primitives['2']
            elif strategy == 3:
                payoffs[profile][f"P{i+1}"] = primitives['3']
            elif strategy == 4:
                payoffs[profile][f"P{i+1}"] = primitives['4']
            else:
                continue
    return {**models, "payoffs": payoffs}


Here are a few simple tests of the payoff primitives for their model.

In [7]:
models = {'payoffs_state': {'strategy_counts': {"2": 2,
                                                "4": 4}},
          'c': 0.5,
          'T': 2,
          'b_r': 4,
          'b_p': 2,
          'r': 0.5,
          'payoffs_key': 'vasconcelos_2014_primitives'}
models = build_payoffs(models)
fastcore.test.test_eq(models['payoff_primitives'],
                      {'1': 4,
                       '2': 2,
                       '3': 2,
                       '4': 1})
models = {**models,
          'payoffs_state': {'strategy_counts': {"2": 0,
                                                "4": 1}}, }
models = build_payoffs(models)
fastcore.test.test_eq(models['payoff_primitives'],
                      {'1': 2,
                       '2': 1,
                       '3': 1,
                       '4': 0.5})


We quickly check that we can generate payoffs for each of the 4**5 possible
interactions in their model.

In [8]:
models = {'c': 0.5,
          'T': 2,
          'b_r': 4,
          'b_p': 2,
          'r': 0.5,
          'payoffs_key': 'vasconcelos_2014'}
models = build_payoffs(models)
fastcore.test.test_eq(len(models['payoffs']), 4**5)


If we are unwilling to use the 5**5 possible strategy profiles for computing
the transition matrices for the evolutionary system, we can always restrict
our attention to the payoffs given the number of contributors from each sector.
We often use hypergeometric sampling anyways when computing the success of
each strategy in the evolutionary system.

## General Payoff Wrapper

In [9]:
# | export payoffs
@method(build_payoffs, 'payoff_function_wrapper')
def build_payoffs(models: dict):
    profiles = create_profiles(models)['profiles']
    profile_payoffs_key = models['profile_payoffs_key']
    payoffs_state = models.get("payoffs_state", {})
    payoffs = {}
    for profile in profiles:
        profile_tuple = string_to_tuple(profile)
        strategy_counts = dict(zip(*np.unique(profile_tuple,
                                              return_counts=True)))
        payoffs_state = {**payoffs_state,
                         'strategy_counts': strategy_counts}
        profile_models = {**models,
                          "strategy_profile": profile,
                          "payoffs_state": payoffs_state,
                          "payoffs_key": profile_payoffs_key}
        profile_payoffs = thread_macro(profile_models,
                                       build_payoffs,
                                       (get, "profile_payoffs"),
                                       )
        payoffs[profile] = {}
        for i, strategy in enumerate(profile_tuple):
            if strategy == 0:
                # A strategy of 0 is reserved for missing players, missing
                # players do not have payoffs.
                continue
            elif str(strategy) in profile_payoffs.keys():
                payoffs[profile][f"P{i+1}"] = profile_payoffs[f"{strategy}"]
            else:
                continue
    return {**models, "payoffs": payoffs}


## Stochastic Payoffs

### Stochastic payoffs

We can compute the payoffs of stochastic games with state-action transition
matrix, $M$, and state-action utilities, $u$, and discount factor, $\delta$,
as follows:

$v = (1 - \delta) v^0 (I - \delta M)^{-1}$ \
$payoffs = v \cdot u$

When $\delta \rightarrow 1$, we instead compute $v$ as the eigenvector of $M$
with associated eigenvalue $1$.

$M$ is the product of a transition matrix and a matrix containing the
probabilities with which each action profile occurs (i.e. a matrix of player
(mixed) strategies). $M$ has size $2mk + 1$, where $m$ is the number of states
and $k$ is the number of strategies available to each player.

We first need to define our flow payoffs, that is, at each state-action
combination, what are the payoffs to each type of player.

In [10]:
# | export payoffs
@method(build_payoffs, "flow_payoffs_wrapper")
def build_payoffs(models):
    "Build the flow payoffs for each state-action in a stochastic game."
    state_actions = models['state_actions']
    payoffs_state = models.get('payoffs_state', {})
    flow_payoffs = collections.defaultdict()
    for state_action in state_actions:
        state, action_profile = str.split(state_action, ":")
        action_tuple = string_to_tuple(action_profile)
        action_counts = dict(zip(*np.unique(action_tuple,
                                            return_counts=True)))
        payoffs_state = {**payoffs_state,
                         'strategy_counts': action_counts,
                         'state': state}
        payoffs_flow_key = models['payoffs_flow_key']
        profile_models = {**models,
                          "payoffs_state": payoffs_state,
                          "payoffs_key": payoffs_flow_key}
        flow_payoffs[state_action] = thread_macro(profile_models,
                                                  build_payoffs,
                                                  (get, "flow_payoffs"),
                                                  )
    return {**models, "flow_payoffs": flow_payoffs}


#| hide

Specify Q as a map of state-action keys to probabilities for each next state \
Specify P as a map of state-action keys to probabilities for each next action that each player could take.

There can be memory issues with storing so many state_actions:
- Total number of possible state_actions is equal to (n_states * n_choices^n_players)^2
- If game is anonymous (so order does not matter), this reduces to (n_states * ncr(n_choices + n_players -1, n_choices-1))^2

The second approach is much much smaller if n_choices is a lot larger than n_players. Unfortunately, we
still need to add up the likelihood of each possible action profile, so computation may still 
be incredibly slow, even if the result still fits in memory.

If we ever find ourselves needing to look at many players when trying to
compute the transition probabilities for a stochastic game, use a monte carlo
simulation instead to learn the transition probabilities and payoffs.

In [11]:
# | export payoffs
@multi
def compute_transition(models):
    "Compute the transition likelihood for the given transition."
    return models.get('compute_transition_key')

@method(compute_transition, 'anonymous_actions')
def compute_transition(models):
    """Compute transition likelihood when we are only passed anonymous action
    profiles (i.e. order does not matter)."""
    P, Q = [models[k] for k in ['P', 'Q']]
    transition_start, transition_end = [models[k] for k in ['transition_start',
                                                            'transition_end']]
    next_state, action_profile = transition_end.split(":")
    action_tuple = string_to_tuple(action_profile)
    action_count = dict(zip(*np.unique(action_tuple, return_counts=True)))
    profiles = create_profiles({**models,
                                "profiles_rule": "from_strategy_count",
                                "strategy_count": action_count})['profiles']
    profile_tuples = map(string_to_tuple, profiles)
    p = [np.prod([P[f"P{player + 1}"][transition_start].get(f"A{action}", 0)
                  for player, action in enumerate(profile_tuple)])
         for profile_tuple in profile_tuples]
    return np.sum(p) * Q[transition_start][next_state]


@method(compute_transition)
def compute_transition(models):
    "Compute transition likelihood given the states and action profiles."
    P, Q = [models[k] for k in ['P', 'Q']]
    transition_start, transition_end = [models[k] for k in ['transition_start',
                                                            'transition_end']]
    next_state, action_profile = transition_end.split(":")
    action_tuple = string_to_tuple(action_profile)
    p = np.prod([P[f"P{player + 1}"][transition_start].get(f"A{action}", 0)
                 for player, action in enumerate(action_tuple)])
    return p * Q[transition_start][next_state]

In [12]:
# | export payoffs
@method(build_payoffs, "stochastic-no-discounting")
def build_payoffs(models: dict):
    """Compute the payoffs for a stochastic game with the given flow_payoffs,
    state_transitions, strategies, and strategy_profile, when there is no
    discounting."""
    u = models['flow_payoffs']
    Q = models['state_transitions']
    strategy_profile = models['strategy_profile'].split("-")[::-1]
    strategies = models['strategies']
    P = {f"P{player + 1}": strategies[strategy_key]
         for player, strategy_key in enumerate(strategy_profile)}
    state_actions = list(Q.keys())
    M = np.zeros((len(state_actions), len(state_actions)))
    for row, transition_start in enumerate(state_actions):
        for col, transition_end in enumerate(state_actions):
            transition_data = {**models,
                               "P": P,
                               "Q": Q,
                               "transition_start": transition_start,
                               "transition_end": transition_end}
            M[row, col] = compute_transition(transition_data)
    v = thread_macro({**models, "transition_matrix": np.array([M])},
                     find_ergodic_distribution,
                     (get, "ergodic"))[0]
    u = np.array([[u[s][f"{i+1}"] for i in range(len(u[s]))]
                  for s in state_actions])
    for _ in range(v.ndim, u.ndim):
        v = v[:, None]
    payoffs = np.sum(v * u, axis=0)
    profile_payoffs = {f"{i+1}": pi for i, pi in enumerate(payoffs)}
    return {**models, "profile_payoffs": profile_payoffs}


@method(build_payoffs, "stochastic-with-discounting")
def build_payoffs(models: dict):
    """Compute the payoffs for a stochastic game with the given flow_payoffs,
    state_transitions, strategies, and strategy_profile."""
    u = models['flow_payoffs']
    Q = models['state_transitions']
    d = models['discount_rate']
    v0 = models['initial_state_action_distribution']
    strategy_profile = models['strategy_profile'].split("-")[::-1]
    strategies = models['strategies']
    P = {f"P{player + 1}": strategies[strategy_key]
         for player, strategy_key in enumerate(strategy_profile)}
    state_actions = list(Q.keys())
    M = np.zeros((len(state_actions), len(state_actions)))
    for row, transition_start in enumerate(state_actions):
        for col, transition_end in enumerate(state_actions):
            transition_data = {**models,
                               "P": P,
                               "Q": Q,
                               "transition_start": transition_start,
                               "transition_end": transition_end}
            M[row, col] = compute_transition(transition_data)
    v = (1 - d) * v0 * np.linalg.inv(np.eye(M.shape) - d * M)
    u = np.array([[u[s][f"{i+1}"] for i in range(len(u[s]))]
                  for s in state_actions])
    for _ in range(v.ndim, u.ndim):
        v = v[:, None]
    payoffs = np.sum(v * u, axis=0)
    profile_payoffs = {f"{i+1}": pi for i, pi in enumerate(payoffs)}
    return {**models, "profile_payoffs": profile_payoffs}


#### Tests for "flow_payoffs_wrapper" method of `build_payoffs`

Here is an example of flow payoffs.

In [13]:
#| export payoffs
@method(build_payoffs, 'vasconcelos_2014_flow')
def build_payoffs(models: dict):
    names = ['payoffs_state', 'c', 'T', 'b_r', 'b_p', 'r', 'g']
    payoffs_state, c, T, b_r, b_p, r, g = [models[k] for k in names]
    strategy_counts = payoffs_state['strategy_counts']
    state = payoffs_state['state']
    reward_bonus = g if state=='1' else 1
    n_r = strategy_counts.get("2", 0)
    n_p = strategy_counts.get("4", 0)
    risk = r * (n_r * c * b_r + n_p * c * b_p < T)
    payoffs = {"1": (1 - risk) * b_r * reward_bonus,  # rich_free_rider
               "2": (1 - risk) * c * b_r * reward_bonus,  # rich_contributor
               "3": (1 - risk) * b_p * reward_bonus,  # poor_free_rider
               "4": (1 - risk) * c * b_p * reward_bonus}  # poor_contributor
    return {**models, "flow_payoffs": payoffs}

In [14]:
models = {"allowed_sectors": {"P1": ["S1", "S2"],
                              "P2": ["S1", "S2"]},
          "sector_strategies": {"S1": [1, 2],
                                "S2": [3, 4]},
          "profiles_rule": "allowed_sectors",}
action_profiles = create_profiles(models)["profiles"]
n_states = 2
state_actions = []
for profile in action_profiles:
    for state in range(n_states):
        state_actions.append(f"{state}:{profile}")

models = {"payoffs_flow_key": "vasconcelos_2014_flow",
          "payoffs_key": "flow_payoffs_wrapper",
          "state_actions": state_actions,
          'c': 0.5,
          'T': 2,
          'b_r': 4,
          'b_p': 2,
          'r': 0.8,
          'g': 2,
          }
flow_payoffs = build_payoffs(models)['flow_payoffs']

In [15]:
flow_payoffs

defaultdict(None,
            {'0:1-1': {'1': 0.7999999999999998,
              '2': 0.3999999999999999,
              '3': 0.3999999999999999,
              '4': 0.19999999999999996},
             '1:1-1': {'1': 1.5999999999999996,
              '2': 0.7999999999999998,
              '3': 0.7999999999999998,
              '4': 0.3999999999999999},
             '0:1-2': {'1': 0.7999999999999998,
              '2': 0.3999999999999999,
              '3': 0.3999999999999999,
              '4': 0.19999999999999996},
             '1:1-2': {'1': 1.5999999999999996,
              '2': 0.7999999999999998,
              '3': 0.7999999999999998,
              '4': 0.3999999999999999},
             '0:1-3': {'1': 0.7999999999999998,
              '2': 0.3999999999999999,
              '3': 0.3999999999999999,
              '4': 0.19999999999999996},
             '1:1-3': {'1': 1.5999999999999996,
              '2': 0.7999999999999998,
              '3': 0.7999999999999998,
              '4': 0.39

#### State transition functions

In [16]:
# | export payoffs
@multi
def state_transition(models):
    "Compute the likelihood of the given state_transition."
    return models.get('state_transition_key')

@method(state_transition, 'ex1')
def state_transition(models):
    """Compute transition likelihood for a model with 2 states and an arbitrary
    number of players. To stay in the good state, 0, all players need to choose
    to cooperate, i.e. action 1."""
    state_action, next_state = [models[k] for k in ['state_action',
                                                    'next_state']]
    current_state, action_profile = state_action.split(":")
    action_tuple = string_to_tuple(action_profile)
    action_count = dict(zip(*np.unique(action_tuple, return_counts=True)))
    n_players = len(action_tuple)
    n_cooperators = action_count.get(1, 0) + action_count.get(3, 0)
    if (current_state == '0'
        and next_state == '1'
        and n_cooperators != n_players):
        transition_likelihood = 1
    elif (current_state == '1'
          and next_state == '0'
          and n_cooperators == n_players):
        transition_likelihood = 1
    elif (current_state == '0'
          and next_state == '0'
          and n_cooperators == n_players):
        transition_likelihood = 1
    elif (current_state == '1'
          and next_state == '1'
          and n_cooperators != n_players):
        transition_likelihood = 1
    else:
        transition_likelihood = 0
    return transition_likelihood

In [17]:
# | export payoffs
def build_state_transitions(models):
    state_actions = models['state_actions']
    n_states = models['n_states']
    state_transitions = {}
    for state_action in state_actions:
        state_transitions[state_action] = {}
        for next_state in [f"{i}" for i in range(n_states)]:
            likelihood = state_transition({**models,
                                           "state_action": state_action,
                                           "next_state": next_state})
            state_transitions[state_action][next_state] = likelihood
    return {**models, "state_transitions": state_transitions}

#### Tests for `build_state_transitions`

In [18]:
models = {"allowed_sectors": {"P1": ["S1", "S2"],
                              "P2": ["S1", "S2"]},
          "sector_strategies": {"S1": [1, 2],
                                "S2": [3, 4]},
          "profiles_rule": "allowed_sectors", }
action_profiles = create_profiles(models)["profiles"]
n_states = 2
state_actions = [f"{state}:{a}"
                 for a in action_profiles
                 for state in range(n_states)]
models = {'n_states':n_states,
          'state_actions': state_actions,
          'state_transition_key': 'ex1'}
result = build_state_transitions(models)['state_transitions']
expected = {'0:1-1': {'0': 1, '1': 0},
 '1:1-1': {'0': 1, '1': 0},
 '0:1-2': {'0': 0, '1': 1},
 '1:1-2': {'0': 0, '1': 1},
 '0:1-3': {'0': 1, '1': 0},
 '1:1-3': {'0': 1, '1': 0},
 '0:1-4': {'0': 0, '1': 1},
 '1:1-4': {'0': 0, '1': 1},
 '0:2-1': {'0': 0, '1': 1},
 '1:2-1': {'0': 0, '1': 1},
 '0:2-2': {'0': 0, '1': 1},
 '1:2-2': {'0': 0, '1': 1},
 '0:2-3': {'0': 0, '1': 1},
 '1:2-3': {'0': 0, '1': 1},
 '0:2-4': {'0': 0, '1': 1},
 '1:2-4': {'0': 0, '1': 1},
 '0:3-1': {'0': 1, '1': 0},
 '1:3-1': {'0': 1, '1': 0},
 '0:3-2': {'0': 0, '1': 1},
 '1:3-2': {'0': 0, '1': 1},
 '0:3-3': {'0': 1, '1': 0},
 '1:3-3': {'0': 1, '1': 0},
 '0:3-4': {'0': 0, '1': 1},
 '1:3-4': {'0': 0, '1': 1},
 '0:4-1': {'0': 0, '1': 1},
 '1:4-1': {'0': 0, '1': 1},
 '0:4-2': {'0': 0, '1': 1},
 '1:4-2': {'0': 0, '1': 1},
 '0:4-3': {'0': 0, '1': 1},
 '1:4-3': {'0': 0, '1': 1},
 '0:4-4': {'0': 0, '1': 1},
 '1:4-4': {'0': 0, '1': 1}}
fastcore.test.test_eq(result, expected)

In [19]:
models = {"allowed_sectors": {"P1": ["S1", "S2"],
                              "P2": ["S1", "S2"]},
          "sector_strategies": {"S1": ["1", "2"],
                                "S2": ["3", "4"]},
          "profiles_rule": "anonymous", }
action_profiles = create_profiles(models)["profiles"]
n_states = 2
state_actions = [f"{state}:{a}"
                 for a in action_profiles
                 for state in range(n_states)]
models = {'n_states':n_states,
          'state_actions': state_actions,
          'state_transition_key': 'ex1'}
result = build_state_transitions(models)['state_transitions']
expected = {'0:4-4': {'0': 0, '1': 1},
 '1:4-4': {'0': 0, '1': 1},
 '0:4-3': {'0': 0, '1': 1},
 '1:4-3': {'0': 0, '1': 1},
 '0:3-3': {'0': 1, '1': 0},
 '1:3-3': {'0': 1, '1': 0},
 '0:4-2': {'0': 0, '1': 1},
 '1:4-2': {'0': 0, '1': 1},
 '0:3-2': {'0': 0, '1': 1},
 '1:3-2': {'0': 0, '1': 1},
 '0:2-2': {'0': 0, '1': 1},
 '1:2-2': {'0': 0, '1': 1},
 '0:4-1': {'0': 0, '1': 1},
 '1:4-1': {'0': 0, '1': 1},
 '0:3-1': {'0': 1, '1': 0},
 '1:3-1': {'0': 1, '1': 0},
 '0:2-1': {'0': 0, '1': 1},
 '1:2-1': {'0': 0, '1': 1},
 '0:1-1': {'0': 1, '1': 0},
 '1:1-1': {'0': 1, '1': 0}}
fastcore.test.test_eq(result, expected)

#### Strategy construction

In [20]:
# | export payoffs
@multi
def build_strategy(models):
    "Build the desired strategy"
    return models.get('strategy_key')

@method(build_strategy, 'ex1_rich_cooperator')
def build_strategy(models):
    """A rich player who cooperates with 95% probability if everyone currently
    cooperates, otherwise defects with 95% probability."""
    state_action = models['state_action']
    current_state, action_profile = state_action.split(":")
    action_tuple = string_to_tuple(action_profile)
    action_count = dict(zip(*np.unique(action_tuple, return_counts=True)))
    n_players = len(action_tuple)
    n_cooperators = action_count.get(1, 0) + action_count.get(3, 0)
    if (current_state == '0'
        and n_cooperators == n_players):
        strategy = {"A1": 0.95, "A2": 0.05}
    elif (current_state == '0'
          and n_cooperators != n_players):
        strategy = {"A1": 0.05, "A2": 0.95}
    elif (current_state == '1'
          and n_cooperators == n_players):
        strategy = {"A1": 0.95, "A2": 0.05}
    elif (current_state == '1'
          and n_cooperators != n_players):
        strategy = {"A1": 0.05, "A2": 0.95}
    return strategy

@method(build_strategy, 'ex1_rich_defector')
def build_strategy(models):
    """A rich player who defects with 95% probability no matter what others
    do, nor what state they are in."""
    state_action = models['state_action']
    current_state, action_profile = state_action.split(":")
    action_tuple = string_to_tuple(action_profile)
    action_count = dict(zip(*np.unique(action_tuple, return_counts=True)))
    n_players = len(action_tuple)
    n_cooperators = action_count.get(1, 0) + action_count.get(3, 0)
    if (current_state == '0'
        and n_cooperators == n_players):
        strategy = {"A1": 0.05, "A2": 0.95}
    elif (current_state == '0'
          and n_cooperators != n_players):
        strategy = {"A1": 0.05, "A2": 0.95}
    elif (current_state == '1'
          and n_cooperators == n_players):
        strategy = {"A1": 0.05, "A2": 0.95}
    elif (current_state == '1'
          and n_cooperators != n_players):
        strategy = {"A1": 0.05, "A2": 0.95}
    return strategy

@method(build_strategy, 'ex1_poor_cooperator')
def build_strategy(models):
    """A poor player who cooperates with 95% probability if everyone currently
    cooperates, otherwise defects with 95% probability."""
    state_action = models['state_action']
    current_state, action_profile = state_action.split(":")
    action_tuple = string_to_tuple(action_profile)
    action_count = dict(zip(*np.unique(action_tuple, return_counts=True)))
    n_players = len(action_tuple)
    n_cooperators = action_count.get(1, 0) + action_count.get(3, 0)
    if (current_state == '0'
        and n_cooperators == n_players):
        strategy = {"A3": 0.95, "A4": 0.05}
    elif (current_state == '0'
          and n_cooperators != n_players):
        strategy = {"A3": 0.05, "A4": 0.95}
    elif (current_state == '1'
          and n_cooperators == n_players):
        strategy = {"A3": 0.95, "A4": 0.05}
    elif (current_state == '1'
          and n_cooperators != n_players):
        strategy = {"A3": 0.05, "A4": 0.95}
    return strategy

@method(build_strategy, 'ex1_poor_defector')
def build_strategy(models):
    """A poor player who defects with 95% probability no matter what others
    do, nor what state they are in."""
    state_action = models['state_action']
    current_state, action_profile = state_action.split(":")
    action_tuple = string_to_tuple(action_profile)
    action_count = dict(zip(*np.unique(action_tuple, return_counts=True)))
    n_players = len(action_tuple)
    n_cooperators = action_count.get(1, 0) + action_count.get(3, 0)
    if (current_state == '0'
        and n_cooperators == n_players):
        strategy = {"A3": 0.05, "A4": 0.95}
    elif (current_state == '0'
          and n_cooperators != n_players):
        strategy = {"A3": 0.05, "A4": 0.95}
    elif (current_state == '1'
          and n_cooperators == n_players):
        strategy = {"A3": 0.05, "A4": 0.95}
    elif (current_state == '1'
          and n_cooperators != n_players):
        strategy = {"A3": 0.05, "A4": 0.95}
    return strategy

In [21]:
# | export payoffs
def build_strategies(models):
    "Build a dictionary containing the specified strategies in `models`"
    state_actions, strategy_keys = [models[k] for k in ["state_actions",
                                                        "strategy_keys"]]
    strategies = {f"{i+1}": {s: build_strategy({"strategy_key": strategy_key,
                                            "state_action": s})
                         for s in state_actions}
              for i, strategy_key in enumerate(strategy_keys)}
    return {**models, "strategies": strategies}

#### Tests for `build_strategy`

In [22]:
models = {"allowed_sectors": {"P1": ["S1", "S2"],
                              "P2": ["S1", "S2"]},
          "sector_strategies": {"S1": ["1", "2"],
                                "S2": ["3", "4"]},
          "profiles_rule": "anonymous", }
action_profiles = create_profiles(models)["profiles"]
n_states = 2
state_actions = [f"{state}:{a}"
                 for a in action_profiles
                 for state in range(n_states)]
strategy_keys = ["ex1_rich_cooperator",
                 "ex1_rich_defector",
                 "ex1_poor_cooperator",
                 "ex1_poor_defector",]
strategies = {f"{i+1}": {s: build_strategy({"strategy_key": strategy_key,
                                            "state_action": s})
                         for s in state_actions}
              for i, strategy_key in enumerate(strategy_keys)}
expected = {'1': {'0:4-4': {'A1': 0.05, 'A2': 0.95},
  '1:4-4': {'A1': 0.05, 'A2': 0.95},
  '0:4-3': {'A1': 0.05, 'A2': 0.95},
  '1:4-3': {'A1': 0.05, 'A2': 0.95},
  '0:3-3': {'A1': 0.95, 'A2': 0.05},
  '1:3-3': {'A1': 0.95, 'A2': 0.05},
  '0:4-2': {'A1': 0.05, 'A2': 0.95},
  '1:4-2': {'A1': 0.05, 'A2': 0.95},
  '0:3-2': {'A1': 0.05, 'A2': 0.95},
  '1:3-2': {'A1': 0.05, 'A2': 0.95},
  '0:2-2': {'A1': 0.05, 'A2': 0.95},
  '1:2-2': {'A1': 0.05, 'A2': 0.95},
  '0:4-1': {'A1': 0.05, 'A2': 0.95},
  '1:4-1': {'A1': 0.05, 'A2': 0.95},
  '0:3-1': {'A1': 0.95, 'A2': 0.05},
  '1:3-1': {'A1': 0.95, 'A2': 0.05},
  '0:2-1': {'A1': 0.05, 'A2': 0.95},
  '1:2-1': {'A1': 0.05, 'A2': 0.95},
  '0:1-1': {'A1': 0.95, 'A2': 0.05},
  '1:1-1': {'A1': 0.95, 'A2': 0.05}},
 '2': {'0:4-4': {'A1': 0.05, 'A2': 0.95},
  '1:4-4': {'A1': 0.05, 'A2': 0.95},
  '0:4-3': {'A1': 0.05, 'A2': 0.95},
  '1:4-3': {'A1': 0.05, 'A2': 0.95},
  '0:3-3': {'A1': 0.05, 'A2': 0.95},
  '1:3-3': {'A1': 0.05, 'A2': 0.95},
  '0:4-2': {'A1': 0.05, 'A2': 0.95},
  '1:4-2': {'A1': 0.05, 'A2': 0.95},
  '0:3-2': {'A1': 0.05, 'A2': 0.95},
  '1:3-2': {'A1': 0.05, 'A2': 0.95},
  '0:2-2': {'A1': 0.05, 'A2': 0.95},
  '1:2-2': {'A1': 0.05, 'A2': 0.95},
  '0:4-1': {'A1': 0.05, 'A2': 0.95},
  '1:4-1': {'A1': 0.05, 'A2': 0.95},
  '0:3-1': {'A1': 0.05, 'A2': 0.95},
  '1:3-1': {'A1': 0.05, 'A2': 0.95},
  '0:2-1': {'A1': 0.05, 'A2': 0.95},
  '1:2-1': {'A1': 0.05, 'A2': 0.95},
  '0:1-1': {'A1': 0.05, 'A2': 0.95},
  '1:1-1': {'A1': 0.05, 'A2': 0.95}},
 '3': {'0:4-4': {'A3': 0.05, 'A4': 0.95},
  '1:4-4': {'A3': 0.05, 'A4': 0.95},
  '0:4-3': {'A3': 0.05, 'A4': 0.95},
  '1:4-3': {'A3': 0.05, 'A4': 0.95},
  '0:3-3': {'A3': 0.95, 'A4': 0.05},
  '1:3-3': {'A3': 0.95, 'A4': 0.05},
  '0:4-2': {'A3': 0.05, 'A4': 0.95},
  '1:4-2': {'A3': 0.05, 'A4': 0.95},
  '0:3-2': {'A3': 0.05, 'A4': 0.95},
  '1:3-2': {'A3': 0.05, 'A4': 0.95},
  '0:2-2': {'A3': 0.05, 'A4': 0.95},
  '1:2-2': {'A3': 0.05, 'A4': 0.95},
  '0:4-1': {'A3': 0.05, 'A4': 0.95},
  '1:4-1': {'A3': 0.05, 'A4': 0.95},
  '0:3-1': {'A3': 0.95, 'A4': 0.05},
  '1:3-1': {'A3': 0.95, 'A4': 0.05},
  '0:2-1': {'A3': 0.05, 'A4': 0.95},
  '1:2-1': {'A3': 0.05, 'A4': 0.95},
  '0:1-1': {'A3': 0.95, 'A4': 0.05},
  '1:1-1': {'A3': 0.95, 'A4': 0.05}},
 '4': {'0:4-4': {'A3': 0.05, 'A4': 0.95},
  '1:4-4': {'A3': 0.05, 'A4': 0.95},
  '0:4-3': {'A3': 0.05, 'A4': 0.95},
  '1:4-3': {'A3': 0.05, 'A4': 0.95},
  '0:3-3': {'A3': 0.05, 'A4': 0.95},
  '1:3-3': {'A3': 0.05, 'A4': 0.95},
  '0:4-2': {'A3': 0.05, 'A4': 0.95},
  '1:4-2': {'A3': 0.05, 'A4': 0.95},
  '0:3-2': {'A3': 0.05, 'A4': 0.95},
  '1:3-2': {'A3': 0.05, 'A4': 0.95},
  '0:2-2': {'A3': 0.05, 'A4': 0.95},
  '1:2-2': {'A3': 0.05, 'A4': 0.95},
  '0:4-1': {'A3': 0.05, 'A4': 0.95},
  '1:4-1': {'A3': 0.05, 'A4': 0.95},
  '0:3-1': {'A3': 0.05, 'A4': 0.95},
  '1:3-1': {'A3': 0.05, 'A4': 0.95},
  '0:2-1': {'A3': 0.05, 'A4': 0.95},
  '1:2-1': {'A3': 0.05, 'A4': 0.95},
  '0:1-1': {'A3': 0.05, 'A4': 0.95},
  '1:1-1': {'A3': 0.05, 'A4': 0.95}}}
fastcore.test.test_eq(strategies, expected)

In [23]:
models = {"allowed_sectors": {"P1": ["S1", "S2"],
                              "P2": ["S1", "S2"]},
          "sector_strategies": {"S1": ["1", "2"],
                                "S2": ["3", "4"]},
          "profiles_rule": "anonymous", }
action_profiles = create_profiles(models)["profiles"]
n_states = 2
state_actions = [f"{state}:{a}"
                 for a in action_profiles
                 for state in range(n_states)]
strategy_keys = ["ex1_rich_cooperator",
                 "ex1_rich_defector",
                 "ex1_poor_cooperator",
                 "ex1_poor_defector",]
models = {**models,
          "strategy_keys": strategy_keys,
          "state_actions": state_actions}
strategies = build_strategies(models)['strategies']
expected = {'1': {'0:4-4': {'A1': 0.05, 'A2': 0.95},
  '1:4-4': {'A1': 0.05, 'A2': 0.95},
  '0:4-3': {'A1': 0.05, 'A2': 0.95},
  '1:4-3': {'A1': 0.05, 'A2': 0.95},
  '0:3-3': {'A1': 0.95, 'A2': 0.05},
  '1:3-3': {'A1': 0.95, 'A2': 0.05},
  '0:4-2': {'A1': 0.05, 'A2': 0.95},
  '1:4-2': {'A1': 0.05, 'A2': 0.95},
  '0:3-2': {'A1': 0.05, 'A2': 0.95},
  '1:3-2': {'A1': 0.05, 'A2': 0.95},
  '0:2-2': {'A1': 0.05, 'A2': 0.95},
  '1:2-2': {'A1': 0.05, 'A2': 0.95},
  '0:4-1': {'A1': 0.05, 'A2': 0.95},
  '1:4-1': {'A1': 0.05, 'A2': 0.95},
  '0:3-1': {'A1': 0.95, 'A2': 0.05},
  '1:3-1': {'A1': 0.95, 'A2': 0.05},
  '0:2-1': {'A1': 0.05, 'A2': 0.95},
  '1:2-1': {'A1': 0.05, 'A2': 0.95},
  '0:1-1': {'A1': 0.95, 'A2': 0.05},
  '1:1-1': {'A1': 0.95, 'A2': 0.05}},
 '2': {'0:4-4': {'A1': 0.05, 'A2': 0.95},
  '1:4-4': {'A1': 0.05, 'A2': 0.95},
  '0:4-3': {'A1': 0.05, 'A2': 0.95},
  '1:4-3': {'A1': 0.05, 'A2': 0.95},
  '0:3-3': {'A1': 0.05, 'A2': 0.95},
  '1:3-3': {'A1': 0.05, 'A2': 0.95},
  '0:4-2': {'A1': 0.05, 'A2': 0.95},
  '1:4-2': {'A1': 0.05, 'A2': 0.95},
  '0:3-2': {'A1': 0.05, 'A2': 0.95},
  '1:3-2': {'A1': 0.05, 'A2': 0.95},
  '0:2-2': {'A1': 0.05, 'A2': 0.95},
  '1:2-2': {'A1': 0.05, 'A2': 0.95},
  '0:4-1': {'A1': 0.05, 'A2': 0.95},
  '1:4-1': {'A1': 0.05, 'A2': 0.95},
  '0:3-1': {'A1': 0.05, 'A2': 0.95},
  '1:3-1': {'A1': 0.05, 'A2': 0.95},
  '0:2-1': {'A1': 0.05, 'A2': 0.95},
  '1:2-1': {'A1': 0.05, 'A2': 0.95},
  '0:1-1': {'A1': 0.05, 'A2': 0.95},
  '1:1-1': {'A1': 0.05, 'A2': 0.95}},
 '3': {'0:4-4': {'A3': 0.05, 'A4': 0.95},
  '1:4-4': {'A3': 0.05, 'A4': 0.95},
  '0:4-3': {'A3': 0.05, 'A4': 0.95},
  '1:4-3': {'A3': 0.05, 'A4': 0.95},
  '0:3-3': {'A3': 0.95, 'A4': 0.05},
  '1:3-3': {'A3': 0.95, 'A4': 0.05},
  '0:4-2': {'A3': 0.05, 'A4': 0.95},
  '1:4-2': {'A3': 0.05, 'A4': 0.95},
  '0:3-2': {'A3': 0.05, 'A4': 0.95},
  '1:3-2': {'A3': 0.05, 'A4': 0.95},
  '0:2-2': {'A3': 0.05, 'A4': 0.95},
  '1:2-2': {'A3': 0.05, 'A4': 0.95},
  '0:4-1': {'A3': 0.05, 'A4': 0.95},
  '1:4-1': {'A3': 0.05, 'A4': 0.95},
  '0:3-1': {'A3': 0.95, 'A4': 0.05},
  '1:3-1': {'A3': 0.95, 'A4': 0.05},
  '0:2-1': {'A3': 0.05, 'A4': 0.95},
  '1:2-1': {'A3': 0.05, 'A4': 0.95},
  '0:1-1': {'A3': 0.95, 'A4': 0.05},
  '1:1-1': {'A3': 0.95, 'A4': 0.05}},
 '4': {'0:4-4': {'A3': 0.05, 'A4': 0.95},
  '1:4-4': {'A3': 0.05, 'A4': 0.95},
  '0:4-3': {'A3': 0.05, 'A4': 0.95},
  '1:4-3': {'A3': 0.05, 'A4': 0.95},
  '0:3-3': {'A3': 0.05, 'A4': 0.95},
  '1:3-3': {'A3': 0.05, 'A4': 0.95},
  '0:4-2': {'A3': 0.05, 'A4': 0.95},
  '1:4-2': {'A3': 0.05, 'A4': 0.95},
  '0:3-2': {'A3': 0.05, 'A4': 0.95},
  '1:3-2': {'A3': 0.05, 'A4': 0.95},
  '0:2-2': {'A3': 0.05, 'A4': 0.95},
  '1:2-2': {'A3': 0.05, 'A4': 0.95},
  '0:4-1': {'A3': 0.05, 'A4': 0.95},
  '1:4-1': {'A3': 0.05, 'A4': 0.95},
  '0:3-1': {'A3': 0.05, 'A4': 0.95},
  '1:3-1': {'A3': 0.05, 'A4': 0.95},
  '0:2-1': {'A3': 0.05, 'A4': 0.95},
  '1:2-1': {'A3': 0.05, 'A4': 0.95},
  '0:1-1': {'A3': 0.05, 'A4': 0.95},
  '1:1-1': {'A3': 0.05, 'A4': 0.95}}}
fastcore.test.test_eq(strategies, expected)

#### Stochastic payoffs test

In [30]:
models = {"allowed_sectors": {"P1": ["S1", "S2"],
                              "P2": ["S1", "S2"]},
          "sector_strategies": {"S1": ["1", "2"],
                                "S2": ["3", "4"]},
          "profiles_rule": "anonymous", }
action_profiles = create_profiles(models)["profiles"]
n_states = 2
state_actions = [f"{state}:{a}"
                 for a in action_profiles
                 for state in range(n_states)]

strategy_keys = ["ex1_rich_cooperator",
                 "ex1_rich_defector",
                 "ex1_poor_cooperator",
                 "ex1_poor_defector",]
models = {**models,
          "strategy_keys": strategy_keys,
          "state_actions": state_actions}
strategies = build_strategies(models)['strategies']
strategy_profile = "1-2-3"
models = {"payoffs_flow_key": "vasconcelos_2014_flow",
          "payoffs_key": "flow_payoffs_wrapper",
          "state_actions": state_actions,
          "strategies": strategies,
          "strategy_profile": strategy_profile,
          'n_states':n_states,
          'state_transition_key': 'ex1',
          'compute_transition_key': "anonymous_actions",
          'c': 0.5,
          'T': 2,
          'b_r': 4,
          'b_p': 2,
          'r': 0.8,
          'g': 2,
          }

models = build_state_transitions(models)
models = build_payoffs(models)
models = {**models,
          "payoffs_key": "stochastic-no-discounting"}
results = build_payoffs(models)
expected = {'1': 1.5979057591623032,
 '2': 0.7989528795811516,
 '3': 0.7989528795811516,
 '4': 0.3994764397905758}
for k, v in results['profile_payoffs'].items():
    fastcore.test.test_close(v, expected[k])

In [31]:
models = {"allowed_sectors": {"P1": ["S1", "S2"],
                              "P2": ["S1", "S2"]},
          "sector_strategies": {"S1": ["1", "2"],
                                "S2": ["3", "4"]},
          "profiles_rule": "anonymous", }
action_profiles = create_profiles(models)["profiles"]
n_states = 2
state_actions = [f"{state}:{a}"
                 for a in action_profiles
                 for state in range(n_states)]

strategy_keys = ["ex1_rich_cooperator",
                 "ex1_rich_defector",
                 "ex1_poor_cooperator",
                 "ex1_poor_defector",]
models = {**models,
          "strategy_keys": strategy_keys,
          "state_actions": state_actions}
strategies = build_strategies(models)['strategies']
strategy_profile = "1-2-3"
models = {**models,
          "payoffs_flow_key": "vasconcelos_2014_flow",
          "payoffs_key": "flow_payoffs_wrapper",
          "state_actions": state_actions,
          "strategies": strategies,
          "strategy_profile": strategy_profile,
          'n_states':n_states,
          'state_transition_key': 'ex1',
          'compute_transition_key': "anonymous_actions",
          'c': 0.5,
          'T': 2,
          'b_r': 4,
          'b_p': 2,
          'r': 0.8,
          'g': 2,
          }
models = build_state_transitions(models)
models = build_payoffs(models)
models = {**models,
          "payoffs_key": "payoff_function_wrapper",
          "profile_payoffs_key": "stochastic-no-discounting"}
results = build_payoffs(models)
results['payoffs']

/mnt/d/Paolo-Work/git/gh-pages-example/gh_pages_example/methods.py:260: ComplexWarning: Casting complex values to real discards the imaginary part
  ergodic = np.array(V.transpose(0, 2, 1)[y], dtype=float)


{'4-4': {'P1': 0.39949999999999986, 'P2': 0.39949999999999986},
 '4-3': {'P1': 0.7989528795811517, 'P2': 0.39947643979057584},
 '3-3': {'P1': 0.7899999999999998, 'P2': 0.7899999999999998},
 '4-2': {'P1': 0.7989999999999997, 'P2': 0.39949999999999986},
 '3-2': {'P1': 0.7989528795811518, 'P2': 0.7989528795811518},
 '2-2': {'P1': 0.7989999999999997, 'P2': 0.7989999999999997},
 '4-1': {'P1': 1.5979057591623036, 'P2': 0.3994764397905759},
 '3-1': {'P1': 1.5799999999999994, 'P2': 0.7899999999999997},
 '2-1': {'P1': 1.5979057591623034, 'P2': 0.7989528795811517},
 '1-1': {'P1': 1.5799999999999994, 'P2': 1.5799999999999994}}

In [32]:
# | hide
import nbdev
nbdev.nbdev_export()
